<a href="https://colab.research.google.com/github/shin497/proofing/blob/main/ReazonSpeech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
"""
注意点
・ランタイムのタブから「ランタイムのタイプを変更」でGPUに設定すると高速に文字起こしができます
・ReazonSpeechに使われるライブラリには依存関係があり、import時にエラーがでる場合があります。
　その場合、ランタイムのタブから「ランタイムを接続解除して削除」をして、再度上から順番に実行すれば動作するかと思います。
　(それでもエラーが出る場合、ちょっと原因はわからないです…すみません…！)
"""

'\n注意点\n・ランタイムのタブから「ランタイムのタイプを変更」でGPUに設定すると高速に文字起こしができます\n・ReazonSpeechに使われるライブラリには依存関係があり、import時にエラーがでる場合があります。\n\u3000その場合、ランタイムのタブから「ランタイムを接続解除して削除」をして、再度上から順番に実行すれば動作するかと思います。\n\u3000(それでもエラーが出る場合、ちょっと原因はわからないです…すみません…！)\n'

In [1]:
%%capture
!pip install git+https://github.com/nyanta012/ReazonSpeech

In [2]:
import torch
from espnet2.bin.asr_inference import Speech2Text
import librosa
import reazonspeech as rs
from google.colab import files

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

speech2text = Speech2Text.from_pretrained(
  "reazon-research/reazonspeech-espnet-next",
  beam_size=5,
  batch_size=0,
  device=device
)

def transcribe(speech, speech2text):
  for cap in rs.transcribe(speech, speech2text):
    print(cap)

# srt file を作成するための関数
def create_file(file_name="transcribe", file_type="srt"):
    if not file_type in ["srt", "txt"]:
        raise NotImplementedError("ファイル拡張子は[srt, txt]を選択してください。")
    with open(f"{file_name}.{file_type}", mode="w", encoding="utf-8") as f:
        for index, _dict in enumerate(rs.transcribe(speech, speech2text)):
            start_time = _dict.start_seconds
            end_time = _dict.end_seconds
            text = _dict.text
            s_h, e_h = int(start_time//(60 * 60)), int(end_time//(60 * 60))
            s_m, e_m = int((start_time % (60 * 60))//60), int((end_time % (60 * 60))//60)
            s_s, e_s = int(start_time % 60), int(end_time % 60)
            if file_type=="srt":
                f.write(f'{index+1}\n{s_h:02}:{s_m:02}:{s_s:02},000 --> {e_h:02}:{e_m:02}:{e_s:02},000\n{text}\n\n')
            elif file_type=="txt":
                f.write(f'[{s_h:02}:{s_m:02}:{s_s:02} ===> {e_h:02}:{e_m:02}:{e_s:02}] {text}\n')

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

In [9]:
uploaded = files.upload()
file_name = list(uploaded)[0]

Saving movie1.mp4 to movie1.mp4


In [10]:
SAMPLE_RATE = 13000
speech, sample_rate = librosa.load(file_name, sr=SAMPLE_RATE)

<ipython-input-10-7776d26a3a24>:2: UserWarning: PySoundFile failed. Trying audioread instead.
  speech, sample_rate = librosa.load(file_name, sr=SAMPLE_RATE)
/usr/local/lib/python3.10/dist-packages/librosa/core/audio.py:184: FutureWarning: librosa.core.audio.__audioread_load
	Deprecated as of librosa version 0.10.0.
	It will be removed in librosa version 1.0.
  y, sr_native = __audioread_load(path, offset, duration, dtype)


In [11]:
create_file(file_type="txt")

/usr/local/lib/python3.10/dist-packages/espnet2/layers/stft.py:164: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  olens = (ilens - self.n_fft) // self.hop_length + 1
